In [32]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import io
import numpy as np
import random
from PIL import Image
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import Callback
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.generic_utils import CustomObjectScope
import tensorflow as tf
from keras.callbacks import EarlyStopping
import keras.backend.tensorflow_backend as tfb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

POS_WEIGHT = 7  # multiplier for positive targets, needs to be tuned

In [2]:
datapath = "/home/kdemochkin/2019_research/UserVisualPreferences/data/AmazonFashion6ImgPartitioned.npy"
[user_train, user_validation, user_test, Item, usernum, itemnum] = np.load(datapath, encoding='bytes')

In [3]:
def prepare_items(items): 
    item_list = []
    for i in range(itemnum):
        item = items[i]
        category = get_categories_from_item(item)
        img = item[b'imgs']
        item_list.append((img, category))
    return item_list

def build_category_dict(items, min_number_of_items_per_category):
    categories = set()
    for i in range(itemnum):
        for cat_list in items[i][b'categories']:
            for cat in cat_list:
                if cat.decode('utf8').lower() not in stop_categories and len(cat.decode('utf8')) > 1: categories.add(cat.decode('utf8').lower())
    categories = dict.fromkeys(list(categories), 0)
    for i in range(itemnum):
        for cat_list in items[i][b'categories']:
            for cat in cat_list:
                if cat.decode('utf8').lower() in categories: categories[cat.decode('utf8').lower()] += 1
    sorted_cats = sorted(categories.items(), key=lambda kv: kv[1], reverse=True)
    top_cats = [c[0] for c in sorted_cats if c[1] >= min_number_of_items_per_category]
    return top_cats, {top_cats[i]: i for i in range(len(top_cats))}

def get_categories_from_item(item):
    item_categories = set()
    for cat_list in item[b'categories']:
        for cat in cat_list:
            if cat.decode('utf8').lower() in categories: item_categories.add(cat.decode('utf8').lower())
    return list(item_categories)

def load_image(img_bytes):
    
    img = preprocess_input(np.array(Image.open(io.BytesIO(img_bytes)).convert('RGB').resize((224, 224))))
    # print(img.shape)
    return img

def cat2idx(categories):
    cat_indices = [category_dict[c] for c in categories]
    return np.array([1 if i in cat_indices else 0 for i in range(num_categories)])

def getitem(idx):
    img, cat = item_list[idx]
    cat = cat2idx(cat)
    return img, cat

In [6]:
stop_categories = ['shoes & accessories: international shipping available',
           'clothing, shoes & jewelry',
           'boys',
           'men',
           'women',
           'girls',
           'clothing',
           'novelty, costumes & more',
           'jewelry: international shipping available',
           'available for snternational shipping']

categories, category_dict = build_category_dict(Item, 1000)
num_categories = len(categories)

In [7]:
combined = prepare_items(Item)
random.shuffle(combined)
combined = combined[:40000]
x, y = zip(*combined)
x = np.array([load_image(img_bytes) for img_bytes in x])
y = np.array([cat2idx(categories) for categories in y])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=105)

In [8]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(36000, 224, 224, 3) (4000, 224, 224, 3) (36000, 75) (4000, 75)


In [9]:
np.save('x_train', x_train)
np.save('x_test', x_test)
np.save('y_train', y_train)
np.save('y_test', y_test)

In [12]:
train_x = x_train
test_x = x_test
train_y = y_train
test_y = y_test

In [ ]:
train_x = np.load("./x_train.npy")
test_x = np.load("./x_test.npy")
train_y = np.load("./y_train.npy")
test_y = np.load("./y_test.npy")

In [ ]:
def chunkIt(seq, num):
    avg = seq.shape[0] / float(num)
    out = []
    last = 0.0

    while last < seq.shape[0]:
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out    

In [ ]:
x_train = np.array(chunkIt(train_x, int(train_x.shape[0] / 64)))
x_test = np.array(chunkIt(test_x, int(test_x.shape[0] / 64)))
y_train = np.array(chunkIt(train_y, int(train_y.shape[0] / 64)))
y_test = np.array(chunkIt(test_y, int(test_y.shape[0] / 64)))

In [ ]:
x_train.shape

In [33]:
class FPRMetrics(Callback):
    def __init__(self, v_x, v_y):
        self.validation_x = v_x
        self.validation_y = v_y

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_x))).round()
        val_targ = self.validation_y
        _val_f1 = f1_score(val_targ, val_predict, average='weighted')
        _val_recall = recall_score(val_targ, val_predict, average='weighted')
        _val_precision = precision_score(val_targ, val_predict, average='weighted')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return


def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor
    and a target tensor. POS_WEIGHT is used as a multiplier
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)



# set all parameters
IM_WIDTH, IM_HEIGHT = 224, 224
BAT_SIZE = 128
NUM_CLASSES = num_categories
NUM_LAYERS_TO_FREEZE = 22  # 3 conv blocks

top_epochs = 10
fit_epochs = 10

top_layers_checkpoint_path = 'res/checkpoint_top_best.hdf5'
fine_tuned_checkpoint_path = 'res/checkpoint_fine_tuned_best.hdf5'
new_extended_mobilenet_weights = 'res/final_weights.hdf5'


# build and transfer learn base model
print('building model')
base_model = MobileNet(
    weights='imagenet',
    include_top=False,
    input_shape=(IM_WIDTH, IM_HEIGHT, 3)
)
top_layer = base_model.output
top_layer = GlobalAveragePooling2D()(top_layer)
top_layer = Dropout(rate=0.5)(top_layer)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(top_layer)


model = Model(inputs=base_model.input, outputs=predictions)

fpr_metrics = FPRMetrics(test_x, test_y)

for layer in model.layers[:NUM_LAYERS_TO_FREEZE]:
    layer.trainable = False
for layer in model.layers[NUM_LAYERS_TO_FREEZE:]:
    layer.trainable = True

mc_top = ModelCheckpoint(
    top_layers_checkpoint_path,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    period=1
)

early_stopping = EarlyStopping(monitor='val_loss', patience=7)

print('transfer learning')
model.compile(
    optimizer=Adam(lr=0.001, decay=0.2),
    loss=weighted_binary_crossentropy,
    metrics=['accuracy']
)
model.fit(
    x=train_x,
    y=train_y,
    batch_size=BAT_SIZE,
    epochs=top_epochs,
    validation_data=(test_x, test_y),
    callbacks=[fpr_metrics, mc_top, early_stopping],
    shuffle=True,
)



# fine tuning
print('setup finetuning')
mc_fit = ModelCheckpoint(
    fine_tuned_checkpoint_path,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    period=1
)

for layer in model.layers[:NUM_LAYERS_TO_FREEZE]:
    layer.trainable = True
for layer in model.layers[NUM_LAYERS_TO_FREEZE:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(lr=0.0001, decay=0.1),
    loss=weighted_binary_crossentropy,
    metrics=['accuracy']
)
print('finetune train')
model.fit(
    x=train_x,
    y=train_y,
    batch_size=BAT_SIZE,
    epochs=fit_epochs,
    callbacks=[fpr_metrics, mc_fit, early_stopping],
    validation_data=(test_x, test_y),
    shuffle=True,
)
print('saving')
model.save(new_extended_mobilenet_weights)

building model
transfer learning
Train on 36000 samples, validate on 4000 samples
Epoch 1/10
36000/36000 [==============================] - 105s 3ms/step - loss: 0.2803 - acc: 0.6876 - val_loss: 0.5850 - val_acc: 0.0670


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 — val_f1: 0.081735 — val_precision: 0.324151 — val_recall 0.097125
Epoch 2/10
36000/36000 [==============================] - 83s 2ms/step - loss: 0.2299 - acc: 0.7188 - val_loss: 0.5736 - val_acc: 0.0785
 — val_f1: 0.087956 — val_precision: 0.324485 — val_recall 0.089738
Epoch 3/10
36000/36000 [==============================] - 84s 2ms/step - loss: 0.2207 - acc: 0.7238 - val_loss: 0.5690 - val_acc: 0.0963
 — val_f1: 0.097788 — val_precision: 0.328518 — val_recall 0.092334
Epoch 4/10
36000/36000 [==============================] - 84s 2ms/step - loss: 0.2161 - acc: 0.7291 - val_loss: 0.5654 - val_acc: 0.1128
 — val_f1: 0.107433 — val_precision: 0.329870 — val_recall 0.099820
Epoch 5/10
36000/36000 [==============================] - 84s 2ms/step - loss: 0.2122 - acc: 0.7310 - val_loss: 0.5659 - val_acc: 0.1072
 — val_f1: 0.101647 — val_precision: 0.333760 — val_recall 0.094330
Epoch 6/10
36000/36000 [==============================] - 84s 2ms/step - loss: 0.2100 - acc: 0.7365 - val_loss: 